In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from copy import deepcopy

from autowoe import ReportDeco, AutoWoE

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("./dataset.csv", low_memory=False)

In [4]:
df.head()

,client_id,age,credit_sum,credit_month,tariff_id,score_shk,living_region,monthly_income,credit_count,overdue_credit_count,...,education_UGR,education_nan,latitude_mean,longitude_mean,durat_ttime_max,durat_ttime_min,durat_ttime_count,durat_ttime_mean,latitude,longitude
0,1,48,59998.00,10,1.6,0.770249,91,30000.0,1.0,1.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,28,10889.00,6,1.1,0.248514,105,43000.0,2.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,32,10728.00,12,1.1,0.459589,156,23000.0,5.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,27,12009.09,12,1.1,0.362536,128,17000.0,2.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,45,16908.89,10,1.1,0.421385,287,25000.0,1.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(["client_id", "open_account_flg"], axis=1), df["open_account_flg"], test_size=0.33, random_state=42)

In [9]:
auto_woe = AutoWoE(n_jobs=1)
auto_woe = ReportDeco(auto_woe)

In [7]:
X_train["TARGET"] = y_train
X_test["TARGET"] = y_test

In [10]:
auto_woe.fit(
    X_train,
    target_name="TARGET",
    validation=X_test
)

[LightGBM] [Info] Number of positive: 16119, number of negative: 75400
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006160 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1885
[LightGBM] [Info] Number of data points in the train set: 91519, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.176127 -> initscore=-1.542809
[LightGBM] [Info] Start training from score -1.542809
age processing...
credit_sum processing...
credit_month processing...
tariff_id processing...
score_shk processing...
living_region processing...
monthly_income processing...
credit_count processing...
overdue_credit_count processing...
gender_F processing...
gender_M processing...
marital_status_CIV processing...
marital_status_DIV processing...
marital_status_MAR processing...
marital_status_UNM processing...
marital_status_WID proce

In [23]:
pred = auto_woe.predict_proba(X_test)

In [24]:
report_params = {"output_path": "./reports", # папка, куда сгенерится отчет и сложатся нужные файлы
                 "report_name": "BASELINE",
                 "report_version_id": 1,
                 "city": "Almaty",
                 "model_name": "credit_model",
                 "zakazchik": "test",
                 "model_aim": "baseline",
                 "model_name": "baseline_logreg",
                 "high_level_department": "1",
                 "ds_name": "1",
                 "target_descr": "1",
                 "non_target_descr": "1"
                }

auto_woe.generate_report(report_params)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 

In [29]:
from sklearn.metrics import f1_score
import numpy as np

def to_labels(pos_probs, threshold):
        return (pos_probs >= threshold).astype('int')
    
def get_optimal_threshold(model, df, step=0.001, booster=False):
    thresholds = np.arange(0, 1, step)
    y = y_test
    probs = model.predict_proba(X_test)
    scores = [f1_score(y, to_labels(probs, t)) for t in thresholds]
    ix = np.argmax(scores)
    print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))
    return thresholds[ix], scores[ix]

In [30]:
threshold, _ = get_optimal_threshold(auto_woe, X_test)

Threshold=0.209, F-Score=0.43189


In [31]:
def save_model(pathmodel, model):
    import pickle

    with open(pathmodel, "wb") as fout:
        pickle.dump(model, fout)

In [54]:
save_model("./models/auto_woe.pkl", auto_woe.model)

In [46]:
test_set = pd.read_csv("dataset_credit_test.csv")

In [2]:
m.predict_proba(test_set)

NameError: name 'm' is not defined

In [50]:
import pickle
def load_model(pathmodel):
    with open(pathmodel, "rb") as fin:
        return pickle.load(fin)

In [55]:
m = load_model("models/auto_woe.pkl")